In [1]:
from carmon import conclusion_pipe
from carmon import common as cmc

Считаем токен из файла

In [2]:
with open("test_data/test_conclusion_pipe/token.txt", "r") as token_file:
    token = token_file.read()

Объявляем токен для дальнейших настроек

In [3]:
stage_1 = conclusion_pipe.state_token(token)

In [4]:
stage_1['success']

True

И проверяем факт авторизации

In [5]:
stage_1['payload']['depart_name']

'НИИ Дезинфектологии Роспотребнадзора'

Запрашиваем доступные на портале заключения

In [6]:
stage_2 = conclusion_pipe.request_possible_conclusions()

In [7]:
stage_2['success']

True

В случае совпадения все ок, в случае несовпадения нужны дополнения словаря. Часть вещей сделана достаточно странно, т.к. на портале, например, `'Iota '` записана именно с пробелом, в противном случае все перестает работать...

In [8]:
stage_2['payload']

{'Alpha': 461014,
 'AT.1': 497466,
 'B.1.1.523': 461012,
 'Beta': 461015,
 'Delta': 461011,
 'Eta ': 9,
 'Gamma': 7,
 'Iota ': 10,
 'Kappa': 6,
 'Lambda': 12,
 'Omicron': 24,
 'Omicron (BA.1)': 38,
 'Omicron (BA.2)': 43,
 'Иной': 4,
 'недостаточное покрытие': 460927}

Считываем таблицу с образцами. Таблица в данном случае представленна с некоторой модификацией, т.е. нельзя сразу подать таблицу, которая получается из `registry_pipe`, так как пока что необходимы дальнейшие модификации. 

Однако общий вид, к которому должна быть сведена таблица, представлен здесь. 

Помимо таблицы с образцами считываются также результаты Pangolin и NextClade. 

In [9]:
stage_3 = conclusion_pipe.read_and_prepare_data("test_data/test_conclusion_pipe/GOOD_TABLE.csv", 
                                                "test_data/test_conclusion_pipe/lineage_report.csv", 
                                                "test_data/test_conclusion_pipe/nextclade.json",
                                                table_separator=",")

In [10]:
stage_3['success']

True

In [11]:
stage_3['payload'].head(3)

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,barcode,pango,nextclade,result,true_id,status
0,104839,1645,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92234,1645,barcode01_MN908947.3,BA.1,21K (Omicron),,,
1,104840,1479,Белгородская область,belg,4798,ФБУЗ ЦГиЭ по Белгородской области,belg140SM89790,1479,barcode02_MN908947.3,BA.1.1,21K (Omicron),,,
2,104841,1660,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92394,1660,barcode03_MN908947.3,BA.1.1,21K (Omicron),,,


После чтения и парсинга всех данных доступны три столбца для заполнения, с ними предстоит разобраться. 

Сперва результаты выставляются локально как комбинация результатов из _pango_ и _nextclade_, после чего, используя локальный словарь, создается результат, соответствующий доступному результату на портале. В случае, если такой результат достичь не удается, выставляется `'NS'`, что в дальнейшем обеспечивает невыставление заключения на портале. 

In [12]:
stage_4 = conclusion_pipe.state_conclusion_local(stage_3['payload'])

In [13]:
stage_4['success']

True

In [14]:
stage_4['payload'].head(3)

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,barcode,pango,nextclade,result,true_id,status
0,104839,1645,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92234,1645,barcode01_MN908947.3,BA.1,21K (Omicron),Omicron (BA.1),,
1,104840,1479,Белгородская область,belg,4798,ФБУЗ ЦГиЭ по Белгородской области,belg140SM89790,1479,barcode02_MN908947.3,BA.1.1,21K (Omicron),Omicron (BA.1),,
2,104841,1660,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92394,1660,barcode03_MN908947.3,BA.1.1,21K (Omicron),Omicron (BA.1),,


После локального выставления результата необходимо запросить истинные _id_ образцов с портала, чтобы в дальнейшем по ним выставить _id_ заключения. 

Предполагается, что все запросы к порталу пройдут успешно и все _id_ будут найдены, в противном случае функция не срабоатет целиком...

In [15]:
stage_5 = conclusion_pipe.request_samples_info(stage_4['payload'])

In [16]:
stage_5['success']

True

In [17]:
stage_5['payload'].head(3)

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,barcode,pango,nextclade,result,true_id,status
0,104839,1645,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92234,1645,barcode01_MN908947.3,BA.1,21K (Omicron),Omicron (BA.1),175152,
1,104840,1479,Белгородская область,belg,4798,ФБУЗ ЦГиЭ по Белгородской области,belg140SM89790,1479,barcode02_MN908947.3,BA.1.1,21K (Omicron),Omicron (BA.1),172708,
2,104841,1660,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92394,1660,barcode03_MN908947.3,BA.1.1,21K (Omicron),Omicron (BA.1),175312,


Используя выставленные _id_, проставим заключения тем образцам, у кого однозначно определено локальное заключение. 

В случае, если запрос прошёл успешно, локально выставляется `'OK'` в поле статуса. В дальнейшем это будет изменено на другое поле, так как можно будет реализовать проверку успеха выставления. 

In [18]:
stage_6 = conclusion_pipe.state_conclusion_remote(stage_5['payload'])

In [19]:
stage_6['success']

True

Тут видно, что части образцов было выставлено значение `'OK'` в поле статуса, т.е. выставление заключения прошло успешно. 

In [20]:
stage_6['payload'].head(3)

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,barcode,pango,nextclade,result,true_id,status
0,104839,1645,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92234,1645,barcode01_MN908947.3,BA.1,21K (Omicron),Omicron (BA.1),175152,OK
1,104840,1479,Белгородская область,belg,4798,ФБУЗ ЦГиЭ по Белгородской области,belg140SM89790,1479,barcode02_MN908947.3,BA.1.1,21K (Omicron),Omicron (BA.1),172708,OK
2,104841,1660,Белгородская область,belg,4875,ФБУЗ ЦГиЭ по Белгородской области,belg140SM92394,1660,barcode03_MN908947.3,BA.1.1,21K (Omicron),Omicron (BA.1),175312,OK


А для образцов с невыясненным заключением проставлено соответствующее замечание в поле статуса. 

In [21]:
df = stage_6['payload']
df[df['status'] != "OK"]

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,barcode,pango,nextclade,result,true_id,status
65,104907,3308,Свердловская область,sver,4477,ФБУЗ ЦГиЭ в Свердловской области,sver131SM72278,3308,barcode66_MN908947.3,None,20A,NS,155100,Unknown conclusion
